## 데이터 전처리 및 저장

In [1]:
!pip3 install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 395.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 55.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 15.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 46.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 40.4 MB/s eta 0:0

In [2]:
import tensorflow as tf
import numpy as np
import pathlib

2024-04-06 04:41:59.723745: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 04:42:01.753478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 데이터 다운로드

In [3]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


### 데이터 전처리

In [4]:
batch_size = 32
img_height = 180
img_width = 180

# 훈련 데이터
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# 검증 데이터
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.


2024-04-06 04:42:38.488937: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Using 734 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### 전처리한 데이터 저장 및 S3에 업로드

In [7]:
!pip3 install -U sagemaker

In [8]:
import os
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
# bucket_name = '[이전에 생성한 버킷 이름]'
bucket_name = 'kmu-m2023807'
print(bucket_name)  

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
kmu-m2023807


In [9]:
# save data locally first
dest = 'flower_model_train_data'
train_path = os.path.join(dest, 'train')
test_path = os.path.join(dest, 'val')

print(train_path)

!mkdir {dest}

train_ds.save(train_path)
val_ds.save(test_path)

flower_model_train_data/train


In [10]:
# upload to S3 bucket (see the bucket name above)
sagemaker_session.upload_data(train_path, bucket=bucket_name, key_prefix='train')
sagemaker_session.upload_data(test_path, bucket=bucket_name, key_prefix='test')

's3://kmu-m2023807/test'

In [11]:
!ls {dest}

train  val


In [12]:
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

In [13]:
role = get_execution_role()
print("execution role ARN:", role)

training_data_uri = os.path.join(f's3://{bucket_name}', 'data')
print(training_data_uri)

execution role ARN: arn:aws:iam::381492271890:role/LabRole
s3://kmu-m2023807/data


In [14]:
num_of_instance = 1                 # 학습에 사용할 인스턴스 개수
instance_type = 'ml.m5.xlarge'     # 학습에 사용할 인스턴스 타입, xlarge 사용

training_script = 'flower_model_train.py'

training_parameters = {
    'epochs': 15, 
    'img_height': img_height, 
    'img_width': img_width
}

# training framework specs
tensorflow_version = '2.14'
python_version = 'py310'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [15]:
# initiate the training job using Tensorflow estimator
flower_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distribution=distributed_training_spec,
    hyperparameters=training_parameters
)

In [16]:
# 15분소요 
flower_estimator.fit(training_data_uri)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-04-06-04-46-24-866


2024-04-06 04:46:26 Starting - Starting the training job...
2024-04-06 04:46:40 Starting - Preparing the instances for training...
2024-04-06 04:47:11 Downloading - Downloading input data......
2024-04-06 04:48:16 Training - Training image download completed. Training in progress..2024-04-06 04:48:23.373364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 04:48:25,712 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-04-06 04:48:25,713 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-06 04:48:25,714 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-04-06 04:48:26,099 sagemaker-training-toolkit INFO     No GPUs 

In [17]:
# 모델 배포

endpoint_name = 'ep-flower-classification'
model_name = 'flower-classification-model'

predictor = flower_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large", # 배포할 인스턴스 타입은 large
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: flower-classification-model
INFO:sagemaker:Creating endpoint-config with name ep-flower-classification
INFO:sagemaker:Creating endpoint with name ep-flower-classification


----!

In [18]:
print(f'endpoint_name= "{endpoint_name}"')
print(f'bucket_name = "{bucket_name}"')

endpoint_name= "ep-flower-classification"
bucket_name = "kmu-m2023807"


### 추론 테스트

In [19]:
from sagemaker.tensorflow import TensorFlowPredictor

In [20]:
flower_model = TensorFlowPredictor(endpoint_name)

In [22]:
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg
!ls

--2024-04-05 09:48:14--  https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.217.91, 142.250.217.123, 142.251.215.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.217.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117948 (115K) [image/jpeg]
Saving to: ‘592px-Red_sunflower.jpg’

100%[======================================>] 117,948     --.-K/s   in 0.02s   

2024-04-05 09:48:15 (6.09 MB/s) - ‘592px-Red_sunflower.jpg’ saved [117948/117948]

1.data.ipynb				      flower_model_train_data
2.train.ipynb				      flower_model_train_deploy.ipynb
592px-Red_sunflower.jpg			      flower_model_train.py
employee_salary_prediction_notebook_kr.ipynb  ncf.py


In [37]:
from tensorflow.keras.preprocessing.image import img_to_array
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

json_serializer = JSONSerializer()
json_deserializer = JSONDeserializer()

flower_model.serializer = json_serializer
flower_model.deserializer = json_deserializer

sunflower_path = "592px-Red_sunflower.jpg"

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)

img_array = img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 

img_array_numpy = img_array.numpy().tolist() 

predictions = flower_model.predict(img_array_numpy)
deserialized_predictions = predictions['predictions']
score = tf.nn.softmax(deserialized_predictions[0])

print("추론 결과: {}".format(score, end='\n\n'))

print("-----------------", end='\n\n')

print("이 꽃의 이름은 {:.2f}% 확률로 {}일 것입니다.".format(100 * np.max(score), class_names[np.argmax(score)]))

추론 결과: [1.1769876e-06 5.5245485e-04 8.5664215e-06 9.9927193e-01 1.6589719e-04]
-----------------

이 꽃의 이름은 99.93% 확률로 sunflowers일 것입니다.


In [ ]:
# delete endpoint at the end of the demo
predictor.delete_endpoint(delete_endpoint_config=True)